# 3.6 Aprendizaje supervisado: Redes neuronales artificiales (Artificial Neural Networks - ANN)

Profesor: Juan Ramón Rico (<juanramonrico@ua.es>)

## Resumen
---

A continuación vamos a presentar los principios básicos de las redes neuronales con ejemplos sobre el MLP (Multilayer Perceptrón). Es una red sencilla donde la neuronas están completamente conectadas, estos es que todas las neuronas de una capa están conectadas con las de la siguiente.

In [56]:
import numpy as np
import pandas as pd

# Cargar datos



In [57]:
## Datos

path = 'https://www.dlsi.ua.es/~juanra/UA/datasets'
path_tennis = f'{path}/tennis-en.csv'
path_covid19 = f'{path}/covid19-en.csv'

## Perceptrón multicapa (MLP): Problema XOR

Este ejemplo está extraído de [Neural Network from scratch in Python](https://towardsdatascience.com/math-neural-network-from-scratch-in-python-d6da9f29ce65)

A continuación vamos a definir la funciones de activación y de pérdida necesarias para construir una red neuronal.

In [58]:
import numpy as np

# TODO: Añadir la función y su derivada para 'sigmoid' y 'relu' con el formato de las siguientes funciones
def sigmoid(x):
  # Función de activación sigmoide
  return 1/(1+np.exp(-x))

def relu(x):
  # Función de activación rectificadora
  return np.maximum(0,x)

def sigmoid_derivative(x):
  # Derivada de la función de activación sigmoide
  return sigmoid(x)*(1-sigmoid(x))

def relu_derivative(x):
  # Derivada de la función de activación rectificadora
  return np.where(x>0,1,0)

def tanh(x):
  # Función de activación tangente hiperbólica
  return np.tanh(x);

def tanh_derivative(x):
  # Derivada de la función de activación tangente hiperbólica
  return 1-np.tanh(x)**2;

def mse(y_true, y_pred):
  # Función de pérdida de error cuadrático medio
  return np.mean((y_true - y_pred) ** 2)

def mse_derivative(y_true, y_pred):
  # Función de pérdida de error cuadrático medio
  return 2*(y_pred - y_true) / len(y_true)

In [59]:
class Layer:
  # Definición de una capa con todas las neuronas conectadas
  def __init__(self, input_size, output_size):
    # input_size = número de neuronas de la capa de entrada
    # output_size = númewo de neuronas de la capa de salida

    self.weights = np.random.rand(input_size, output_size) - 0.5
    self.bias = np.random.rand(1, output_size) - 0.5

  def forward_propagation(self, input_data):
    # Calcula la salida (output) para una entreda dada (input_data)
    self.input = input_data
    self.output = np.dot(self.input, self.weights) + self.bias
    return self.output

  def backward_propagation(self, output_error, learning_rate):
    # Calcula la retropropagación de errores mediante las derivadas
    # dE/dW y dE/dB a partir de un error de salida dado (output_error=dE/dY)
    # Devuelve el error de entrada (input_error=dE/dX).

    #: calcular el error cometido por esta capa de la salida respecto de los pesos actuales
    input_error = np.dot(output_error, self.weights.T)
    
    #: calcular el error cometido por esta capa de la entrada respecto del error de salida
    weights_error = np.dot(self.input.T, output_error)

    # Actualización de los pesos y los bias (dBias = output_error)
    self.weights -= learning_rate * weights_error 
    self.bias -= learning_rate * output_error

    return input_error


class ActivationLayer(Layer):
  # Función de activación y su derivada para una capa
  def __init__(self, activation, activation_derivative):
    # Asignación de la función de activación y su derivada

    self.activation = activation
    self.activation_derivative = activation_derivative

  def forward_propagation(self, input_data):
    # Devuelve el cálculo de aplicar la función de activiación
    # a una entrada (input_data)

    self.input = input_data
    self.output = self.activation(self.input)
    return self.output

  def backward_propagation(self, output_error, learning_rate):
    # Devuelve la derivada, input_error=dE/dX, para un error de salida, output_error=dE/dY.
    # No usamos learning_rate porque no es un parámetro que se tenga que "aprender".
    return self.activation_derivative(self.input) * output_error


class Network:
  # Perceptrón Multicapa - MLP
  def __init__(self):
    self.layers = []
    self.loss = None
    self.loss_derivative = None

  def add(self, layer):
    # Añadir una capa oculta nueva
    self.layers.append(layer)

  def set_loss(self, loss, loss_derivative):
    # Asignar las funciones de pérdida y su derivada
    self.loss = loss
    self.loss_derivative = loss_derivative

  def predict(self, input_data):
    # Predicción de un salida para una entrada dada

    # Tamño del la entrada
    n_samples = len(input_data)
    result = []

    for i in range(n_samples):
      # Aplicar la red a todos los ejemplos

      # Propagación hacia adelante (forward propagation)
      output = input_data[i]
      for layer in self.layers:
          output = layer.forward_propagation(output)
      result.append(output)

    return np.stack(result)

  def fit(self, x_train, y_train, epochs, learning_rate, verbose=1):
    # Entrenamiento de la red (fit)

    # Tamaño de la entrada
    n_samples = len(x_train)

    for i in range(epochs):
      # Bucle de entrenamiento

      err = 0
      for j in range(n_samples):
        # Propagación hacia adelante (forward propagation)
        output = np.expand_dims(x_train[j], axis=0)
        for layer in self.layers:
          output = layer.forward_propagation(output)

        # Calcular la pérdida (solo para mostrarla)
        err += self.loss(y_train[j], output)

        # Retropropagación (backward propagation)
        error = self.loss_derivative(y_train[j], output)
        for layer in reversed(self.layers):
            error = layer.backward_propagation(error, learning_rate)

      # calculate average error on all samples
      err /= n_samples
      if verbose>0 and (i+1) % 100 == 0:
        print(f'epoch: {i+1}/{epochs}  losss: {err:.5f}')

In [60]:
np.set_printoptions(suppress=True)  # Forzar a numpy a mostrar formato sin exponente
np.random.seed(1)                   # Inciar semilla pseudo-aleatoria para verificar los cálculos

# Conjunto de entrenamiento
X_train = np.array([[0,0], [0,1], [1,0], [1,1]])
y_train = np.array([[0], [1], [1], [0]])

print('X_train.shape', X_train.shape, 'y_train.shape',y_train.shape)

# Red neuronal
model = Network()
model.add(Layer(2, 3))
model.add(ActivationLayer(tanh, tanh_derivative))

model.add(Layer(3, 1))
model.add(ActivationLayer(tanh, tanh_derivative))

# Entrenamiento
model.set_loss(mse, mse_derivative)
model.fit(X_train, y_train, epochs=1000, learning_rate=0.1)

# Test
df_out = pd.DataFrame({'y_pred':model.predict(X_train).ravel(), 'y_true':y_train.ravel()}).round(4)
print(df_out)

'''
X_train.shape (4, 2) y_train.shape (4, 1)
epoch: 100/1000  losss: 0.20690
epoch: 200/1000  losss: 0.08369
epoch: 300/1000  losss: 0.00355
epoch: 400/1000  losss: 0.00145
epoch: 500/1000  losss: 0.00088
epoch: 600/1000  losss: 0.00062
epoch: 700/1000  losss: 0.00047
epoch: 800/1000  losss: 0.00038
epoch: 900/1000  losss: 0.00032
epoch: 1000/1000  losss: 0.00027
   y_pred  y_true
0  0.0009       0
1  0.9759       1
2  0.9778       1
3 -0.0017       0
'''

X_train.shape (4, 2) y_train.shape (4, 1)
epoch: 100/1000  losss: 0.20690
epoch: 200/1000  losss: 0.08369
epoch: 300/1000  losss: 0.00355
epoch: 400/1000  losss: 0.00145
epoch: 500/1000  losss: 0.00088
epoch: 600/1000  losss: 0.00062
epoch: 700/1000  losss: 0.00047
epoch: 800/1000  losss: 0.00038
epoch: 900/1000  losss: 0.00032
epoch: 1000/1000  losss: 0.00027
   y_pred  y_true
0  0.0009       0
1  0.9759       1
2  0.9778       1
3 -0.0017       0


'\nX_train.shape (4, 2) y_train.shape (4, 1)\nepoch: 100/1000  losss: 0.20690\nepoch: 200/1000  losss: 0.08369\nepoch: 300/1000  losss: 0.00355\nepoch: 400/1000  losss: 0.00145\nepoch: 500/1000  losss: 0.00088\nepoch: 600/1000  losss: 0.00062\nepoch: 700/1000  losss: 0.00047\nepoch: 800/1000  losss: 0.00038\nepoch: 900/1000  losss: 0.00032\nepoch: 1000/1000  losss: 0.00027\n   y_pred  y_true\n0  0.0009       0\n1  0.9759       1\n2  0.9778       1\n3 -0.0017       0\n'

## Ejercicio 1

- Modifica el ejemplo anterior para usar las funciones de activación:
  - `sigmoidea` y
  - `relu`
- Comprueba con varias ejecuciones a partir de qué número de épocas el algoritmo comienza a aprender modificando el número total de épocas y el ratio de aprendizaje, `learning_rate` (ej. valores de $error<0.1$).

In [22]:
# Solución enp.set_printoptions(suppress=True)  # Forzar a numpy a mostrar formato sin exponente
np.random.seed(1)                   # Inciar semilla pseudo-aleatoria para verificar los cálculos

# Conjunto de entrenamiento
X_train = np.array([[0,0], [0,1], [1,0], [1,1]])
y_train = np.array([[0], [1], [1], [0]])

print('X_train.shape', X_train.shape, 'y_train.shape',y_train.shape)

# Red neuronal
model = Network()
model.add(Layer(2, 3))
model.add(ActivationLayer(sigmoid, sigmoid_derivative))

model.add(Layer(3, 1))
model.add(ActivationLayer(sigmoid, sigmoid_derivative))

# Entrenamiento
model.set_loss(mse, mse_derivative)
model.fit(X_train, y_train, epochs=2000, learning_rate=0.2)

# Test
df_out = pd.DataFrame({'y_pred':model.predict(X_train).ravel(), 'y_true':y_train.ravel()}).round(4)
print(df_out)


X_train.shape (4, 2) y_train.shape (4, 1)
epoch: 100/2000  losss: 0.25948
epoch: 200/2000  losss: 0.25919
epoch: 300/2000  losss: 0.25893
epoch: 400/2000  losss: 0.25867
epoch: 500/2000  losss: 0.25840
epoch: 600/2000  losss: 0.25809
epoch: 700/2000  losss: 0.25765
epoch: 800/2000  losss: 0.25687
epoch: 900/2000  losss: 0.25515
epoch: 1000/2000  losss: 0.25084
epoch: 1100/2000  losss: 0.24064
epoch: 1200/2000  losss: 0.22159
epoch: 1300/2000  losss: 0.19963
epoch: 1400/2000  losss: 0.18221
epoch: 1500/2000  losss: 0.16772
epoch: 1600/2000  losss: 0.15241
epoch: 1700/2000  losss: 0.12694
epoch: 1800/2000  losss: 0.08895
epoch: 1900/2000  losss: 0.05849
epoch: 2000/2000  losss: 0.03934
   y_pred  y_true
0  0.1242       0
1  0.8043       1
2  0.8167       1
3  0.2569       0


'\nX_train.shape (4, 2) y_train.shape (4, 1)\nepoch: 100/1000  losss: 0.20690\nepoch: 200/1000  losss: 0.08369\nepoch: 300/1000  losss: 0.00355\nepoch: 400/1000  losss: 0.00145\nepoch: 500/1000  losss: 0.00088\nepoch: 600/1000  losss: 0.00062\nepoch: 700/1000  losss: 0.00047\nepoch: 800/1000  losss: 0.00038\nepoch: 900/1000  losss: 0.00032\nepoch: 1000/1000  losss: 0.00027\n   y_pred  y_true\n0  0.0009       0\n1  0.9759       1\n2  0.9778       1\n3 -0.0017       0\n'

In [36]:
# Solución enp.set_printoptions(suppress=True)  # Forzar a numpy a mostrar formato sin exponente
np.random.seed(1)                   # Inciar semilla pseudo-aleatoria para verificar los cálculos

# Conjunto de entrenamiento
X_train = np.array([[0,0], [0,1], [1,0], [1,1]])
y_train = np.array([[0], [1], [1], [0]])

print('X_train.shape', X_train.shape, 'y_train.shape',y_train.shape)

# Red neuronal
model = Network()
model.add(Layer(2, 3))
model.add(ActivationLayer(relu, relu_derivative))

model.add(Layer(3, 1))
model.add(ActivationLayer(relu, relu_derivative))  # Cambio a sigmoidea en la capa de salida

# Entrenamiento
model.set_loss(mse, mse_derivative)
model.fit(X_train, y_train, epochs=2000, learning_rate=0.6)  # Ajusta la tasa de aprendizaje si es necesario

# Test
df_out = pd.DataFrame({'y_pred': model.predict(X_train).ravel(), 'y_true': y_train.ravel()}).round(4)
print(df_out)



X_train.shape (4, 2) y_train.shape (4, 1)
epoch: 100/2000  losss: 0.50000
epoch: 200/2000  losss: 0.50000
epoch: 300/2000  losss: 0.50000
epoch: 400/2000  losss: 0.50000
epoch: 500/2000  losss: 0.50000
epoch: 600/2000  losss: 0.50000
epoch: 700/2000  losss: 0.50000
epoch: 800/2000  losss: 0.50000
epoch: 900/2000  losss: 0.50000
epoch: 1000/2000  losss: 0.50000
epoch: 1100/2000  losss: 0.50000
epoch: 1200/2000  losss: 0.50000
epoch: 1300/2000  losss: 0.50000
epoch: 1400/2000  losss: 0.50000
epoch: 1500/2000  losss: 0.50000
epoch: 1600/2000  losss: 0.50000
epoch: 1700/2000  losss: 0.50000
epoch: 1800/2000  losss: 0.50000
epoch: 1900/2000  losss: 0.50000
epoch: 2000/2000  losss: 0.50000
   y_pred  y_true
0     0.0       0
1     0.0       1
2     0.0       1
3     0.0       0


## Ejercicio 2

Adapta los datos de `tenis` para que funcione correctamente con la red de ejemplo.
- Tendrás que convertir las etiquetas de cada categoría a valores numéricos.
- Si la categoría es binaria, ej. Sí/No se puede convertir a 1/0;
- Si la variable tiene más de dos categorías se puede representar con el formato llamado `one hot` que consiste en crear un vector de ceros de tamaño número de categorías, y un solo uno en la posición de la etiqueta que queremos representar. Ej. `[sunny, overcast, rain]` y queremos representar `overcast` sería con `[0,1,0]`.
- Revisad la documentación del paquete `Pandas` y la función `get_dummies()`.

Una vez adaptada los datos para la entrada probaremos diferentes configuraciones como:
- Número de capas ocultas 1 o 2;
- Número de neuronas por capa 4, 8 o 16;
- Funcion de activiación `sigmoidea`, `tanh` o `relu`.

In [50]:
# Datos sobre Tenis
data = pd.read_csv(path_tennis)
data.head()

,weather,temperature,humidity,wind,play
0,sunny,high,high,no,no
1,sunny,high,high,yes,no
2,overcast,high,high,no,yes
3,rain,medium,high,no,yes
4,rain,low,normal,no,yes


In [51]:
# TODO: Adaptar los datos para la entrada (X) y salida (y) del algoritmo
# Codificación one-hot para las columnas con más de dos categorías
data_encoded = pd.get_dummies(data, columns=['weather', 'temperature', 'humidity', 'wind', 'play']).astype(int)

X = data_encoded.drop(['play_no', 'play_yes', 'humidity_high','wind_no' ], axis=1).values
y = data_encoded[['play_yes']].values


In [55]:
np.set_printoptions(suppress=True)  # Forzar a numpy a mostrar formato sin exponente
np.random.seed(1)                   # Inciar semilla pseudo-aleatoria para verificar los cálculos

# Red neuronal
model = Network()
model.add(Layer(8, 16))
#model.add(ActivationLayer(tanh, tanh_derivative))
#model.add(ActivationLayer(sigmoid, sigmoid_derivative))
model.add(ActivationLayer(relu, relu_derivative))
model.add(Layer(16, 1))
#model.add(ActivationLayer(tanh, tanh_derivative))
#model.add(ActivationLayer(sigmoid, sigmoid_derivative))
model.add(ActivationLayer(relu, relu_derivative))

# Entrenamiento
model.set_loss(mse, mse_derivative)
model.fit(X, y, epochs=1000, learning_rate=0.1)

# Test
y_pred = model.predict(X).ravel()
y_true = y.ravel()
acc = (y_pred>0.5) == y_true
df_out = pd.DataFrame({'y_pred':y_pred, 'y_true':y_true, 'acc':acc}).round(2)

print()
print(df_out)


epoch: 100/1000  losss: 0.64286
epoch: 200/1000  losss: 0.64286
epoch: 300/1000  losss: 0.64286
epoch: 400/1000  losss: 0.64286
epoch: 500/1000  losss: 0.64286
epoch: 600/1000  losss: 0.64286
epoch: 700/1000  losss: 0.64286
epoch: 800/1000  losss: 0.64286
epoch: 900/1000  losss: 0.64286
epoch: 1000/1000  losss: 0.64286

    y_pred  y_true    acc
0      0.0       0   True
1      0.0       0   True
2      0.0       1  False
3      0.0       1  False
4      0.0       1  False
5      0.0       0   True
6      0.0       1  False
7      0.0       0   True
8      0.0       1  False
9      0.0       1  False
10     0.0       1  False
11     0.0       1  False
12     0.0       1  False
13     0.0       0   True


## Ejercicio 3

Adapta los datos de `covid19` para que funcione correctamente con la red de ejemplo. Puedes aplicar transformaciones similares a las del ejercicio anterior.

Una vez adaptada la entreda probamos diferentes configuraciones de red como:
- Número de capas ocultas 1 o 2;
- Número de neuronas por capa 4,8 o 16;
- Funcion de activiación `sigmoidea`, `tanh` o `relu`.

In [62]:
# Datos sobre Covid-19
data = pd.read_csv(path_covid19, index_col=0)
data.head()

,Fever,Cough,Respiratory Problems,Infected
Id,,,,
1,No,No,No,No
2,Yes,Yes,Yes,Yes
3,Yes,Yes,No,No
4,Yes,No,Yes,Yes
5,Yes,Yes,Yes,Yes


In [75]:
# TODO: Adaptar los datos para la entrada (X) y salida (y) del algoritmo
data_encoded = pd.get_dummies(data, columns=['Fever', 'Cough', 'Respiratory Problems', 'Infected']).astype(int)
print(data_encoded.head())
X = data_encoded.drop(['Infected_No', 'Infected_Yes', 'Fever_No', 'Cough_No','Respiratory Problems_No' ], axis=1).values
y = data_encoded[['Infected_Yes']].values


    Fever_No  Fever_Yes  Cough_No  Cough_Yes  Respiratory Problems_No  \
Id                                                                      
1          1          0         1          0                        1   
2          0          1         0          1                        0   
3          0          1         0          1                        1   
4          0          1         1          0                        0   
5          0          1         0          1                        0   

    Respiratory Problems_Yes  Infected_No  Infected_Yes  
Id                                                       
1                          0            1             0  
2                          1            0             1  
3                          0            1             0  
4                          1            0             1  
5                          1            0             1  


In [76]:
np.set_printoptions(suppress=True)  # Forzar a numpy a mostrar formato sin exponente
np.random.seed(1)                   # Inciar semilla pseudo-aleatoria para verificar los cálculos

# Red neuronal
model = Network()
model.add(Layer(3, 16))
#model.add(ActivationLayer(tanh, tanh_derivative))
model.add(ActivationLayer(sigmoid, sigmoid_derivative))
#model.add(ActivationLayer(relu, relu_derivative))
model.add(Layer(16, 1))
#model.add(ActivationLayer(tanh, tanh_derivative))
model.add(ActivationLayer(sigmoid, sigmoid_derivative))
#model.add(ActivationLayer(relu, relu_derivative))

# Entrenamiento
model.set_loss(mse, mse_derivative)
model.fit(X, y, epochs=1000, learning_rate=0.1)

# Test
y_pred = model.predict(X).ravel()
y_true = y.ravel()
acc = (y_pred>0.5) == y_true
df_out = pd.DataFrame({'y_pred':y_pred, 'y_true':y_true, 'acc':acc}).round(2)

print()
print(df_out)

'''
epoch: 100/1000  losss: 0.12176
epoch: 200/1000  losss: 0.11990
epoch: 300/1000  losss: 0.11836
epoch: 400/1000  losss: 0.11689
epoch: 500/1000  losss: 0.11595
epoch: 600/1000  losss: 0.11556
epoch: 700/1000  losss: 0.11539
epoch: 800/1000  losss: 0.11530
epoch: 900/1000  losss: 0.11524
epoch: 1000/1000  losss: 0.11520

    y_pred  y_true    acc
0     0.02       0   True
1     0.93       1   True
2     0.29       0   True
3     0.95       1   True
4     0.93       1   True
5    -0.09       0   True
6     0.95       1   True
7     0.95       1   True
8     0.68       1   True
9     0.29       1  False
10   -0.09       0   True
11    0.68       1   True
12    0.68       0  False
13    0.29       0   True
'''

epoch: 100/1000  losss: 0.11415
epoch: 200/1000  losss: 0.10464
epoch: 300/1000  losss: 0.10347
epoch: 400/1000  losss: 0.10305
epoch: 500/1000  losss: 0.10282
epoch: 600/1000  losss: 0.10265
epoch: 700/1000  losss: 0.10251
epoch: 800/1000  losss: 0.10238
epoch: 900/1000  losss: 0.10225
epoch: 1000/1000  losss: 0.10213

    y_pred  y_true    acc
0     0.04       0   True
1     0.96       1   True
2     0.32       0   True
3     0.98       1   True
4     0.96       1   True
5     0.02       0   True
6     0.98       1   True
7     0.98       1   True
8     0.66       1   True
9     0.32       1  False
10    0.02       0   True
11    0.66       1   True
12    0.66       0  False
13    0.32       0   True


'\nepoch: 100/1000  losss: 0.12176\nepoch: 200/1000  losss: 0.11990\nepoch: 300/1000  losss: 0.11836\nepoch: 400/1000  losss: 0.11689\nepoch: 500/1000  losss: 0.11595\nepoch: 600/1000  losss: 0.11556\nepoch: 700/1000  losss: 0.11539\nepoch: 800/1000  losss: 0.11530\nepoch: 900/1000  losss: 0.11524\nepoch: 1000/1000  losss: 0.11520\n\n    y_pred  y_true    acc\n0     0.02       0   True\n1     0.93       1   True\n2     0.29       0   True\n3     0.95       1   True\n4     0.93       1   True\n5    -0.09       0   True\n6     0.95       1   True\n7     0.95       1   True\n8     0.68       1   True\n9     0.29       1  False\n10   -0.09       0   True\n11    0.68       1   True\n12    0.68       0  False\n13    0.29       0   True\n'

# MLP (scikit-learn)

Vamos a utilizar las funciones sobre perceptrones multicapa del paquete `scikit-learn`. Concretamente usaremos la función de `MLPRegressor()` para entrenar y predecir.

A continuación tenéis un ejemplo con los datos de la función XOR.

In [77]:
import numpy as np
from sklearn.neural_network import MLPRegressor

# Creamos un dataset de ejemplo
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0])

# Creamos una instancia del modelo secuencial
model = MLPRegressor(hidden_layer_sizes=8, activation='tanh', max_iter=10000, random_state=0)

# Entrenamos el modelo
model.fit(X, y)

# Evaluamos el modelo
y_pred = model.predict(X).ravel()
y_true = y.ravel()
acc = (y_pred>0.5) == y_true
df_out = pd.DataFrame({'y_pred':y_pred, 'y_true':y_true, 'acc':acc}).round(2)

print(df_out)

   y_pred  y_true   acc
0    0.12       0  True
1    0.87       1  True
2    0.91       1  True
3    0.11       0  True


## Ejercicio 4

Modifica el ejemplo anterior de `MLPRegressor()` para entrenar y calcular los resultados de los conjuntos de datos de `tenis` y `covid19`.

### Tenis

In [78]:
import numpy as np
from sklearn.neural_network import MLPRegressor

data = pd.read_csv(path_tennis)

# TODO: Adaptar los datos para la entrada (X) y salida (y) del algoritmo

data_encoded = pd.get_dummies(data, columns=['weather', 'temperature', 'humidity', 'wind', 'play']).astype(int)

X = data_encoded.drop(['play_no', 'play_yes', 'humidity_high','wind_no' ], axis=1).values
y = data_encoded[['play_yes']].values

# Creamos una instancia del modelo secuencial
model = MLPRegressor(hidden_layer_sizes=16, activation='logistic', max_iter=1000, random_state=0)

# Entrenamos el modelo
model.fit(X, y)

# Evaluamos el modelo
# Test
# Test
y_pred = model.predict(X).ravel()
y_true = y.ravel()
acc = (y_pred>0.5) == y_true
df_out = pd.DataFrame({'y_pred':y_pred, 'y_true':y_true, 'acc':acc}).round(2)

print()
print(df_out)

'''
X.shape (14, 8) y.shape (14,)

    y_pred  y_true    acc
0     0.24       0   True
1     0.00       0   True
2     0.74       1   True
3     0.62       1   True
4     0.79       1   True
5     0.52       0  False
6     0.98       1   True
7     0.57       0  False
8     0.74       1   True
9     0.95       1   True
10    0.62       1   True
11    0.81       1   True
12    1.07       1   True
'''


    y_pred  y_true    acc
0     0.25       0   True
1    -0.02       0   True
2     0.80       1   True
3     0.66       1   True
4     0.80       1   True
5     0.50       0  False
6     1.00       1   True
7     0.61       0  False
8     0.75       1   True
9     0.88       1   True
10    0.54       1   True
11    0.87       1   True
12    1.02       1   True
13    0.37       0   True


/Users/daniel/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


'\nX.shape (14, 8) y.shape (14,)\n\n    y_pred  y_true    acc\n0     0.24       0   True\n1     0.00       0   True\n2     0.74       1   True\n3     0.62       1   True\n4     0.79       1   True\n5     0.52       0  False\n6     0.98       1   True\n7     0.57       0  False\n8     0.74       1   True\n9     0.95       1   True\n10    0.62       1   True\n11    0.81       1   True\n12    1.07       1   True\n'

### COVID-19

In [79]:
import numpy as np
from sklearn.neural_network import MLPRegressor

data = pd.read_csv(path_covid19)

# TODO: Adaptar los datos para la entrada (X) y salida (y) del algoritmo
data_encoded = pd.get_dummies(data, columns=['Fever', 'Cough', 'Respiratory Problems', 'Infected']).astype(int)
print(data_encoded.head())
X = data_encoded.drop(['Infected_No', 'Infected_Yes', 'Fever_No', 'Cough_No','Respiratory Problems_No' ], axis=1).values
y = data_encoded[['Infected_Yes']].values


# Creamos una instancia del modelo secuencial
model = MLPRegressor(hidden_layer_sizes=16, activation='logistic', max_iter=1000, random_state=0)

# Entrenamos el modelo
model.fit(X, y)

# Evaluamos el modelo
# Test
# Test
y_pred = model.predict(X).ravel()
y_true = y.ravel()
acc = (y_pred>0.5) == y_true
df_out = pd.DataFrame({'y_pred':y_pred, 'y_true':y_true, 'acc':acc}).round(2)

print()
print(df_out)

'''
.shape (14, 4) y.shape (14,)

    y_pred  y_true    acc
0     0.32       0   True
1     0.52       1   True
2     0.50       0  False
3     0.54       1   True
4     0.60       1   True
5     0.51       0  False
6     0.60       1   True
7     0.61       1   True
8     0.59       1   True
9     0.61       1   True
10    0.55       0  False
11    0.60       1   True
12    0.60       0  False
13    0.62       0  False
'''

   Id  Fever_No  Fever_Yes  Cough_No  Cough_Yes  Respiratory Problems_No  \
0   1         1          0         1          0                        1   
1   2         0          1         0          1                        0   
2   3         0          1         0          1                        1   
3   4         0          1         1          0                        0   
4   5         0          1         0          1                        0   

   Respiratory Problems_Yes  Infected_No  Infected_Yes  
0                         0            1             0  
1                         1            0             1  
2                         0            1             0  
3                         1            0             1  
4                         1            0             1  

    y_pred  y_true    acc
0     0.32       0   True
1     0.52       1   True
2     0.50       0  False
3     0.54       1   True
4     0.60       1   True
5     0.51       0  False
6     0.60       1

/Users/daniel/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


'\n.shape (14, 4) y.shape (14,)\n\n    y_pred  y_true    acc\n0     0.32       0   True\n1     0.52       1   True\n2     0.50       0  False\n3     0.54       1   True\n4     0.60       1   True\n5     0.51       0  False\n6     0.60       1   True\n7     0.61       1   True\n8     0.59       1   True\n9     0.61       1   True\n10    0.55       0  False\n11    0.60       1   True\n12    0.60       0  False\n13    0.62       0  False\n'